<a href="https://colab.research.google.com/github/nkampel/data_science_tutorial/blob/main/data_science_tutorial_may_5_HPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference
Notebook from: https://www.kaggle.com/code/mohamedchahed/brain-tumor-mri-classification-with-vgg19

Modified for Datascience practical Presentation

Original Author: MOHAMED CHAHED

## This Notebook is an example for excecuting Hyperparameter Optimization (HPO) by using the cloud service "Weights&Biases"
https://wandb.ai/site

The beginning of this notebook is mostly identical with "data_science_tutorial_may_5.ipynb"
In the lower part is an example for intergrating your training with "W&B sweeps", aka. HPO

Installing package:

In [ ]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importing packages

In [ ]:
# Import necessary libraries 
import os
import numpy as np
import cv2 
import imutils
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

# Dataset Download

**Download** the dataset:
https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection

In [ ]:
!git clone https://github.com/nkampel/data_science_tutorial.git;
!unzip -o -qq data_science_tutorial/archive.zip;

fatal: destination path 'data_science_tutorial' already exists and is not an empty directory.


# Cleaning Data

We found that there where mislabled images in the dataset. So we  manualy, deleted them from the directory!

In [ ]:
!rm no/N1.JPG no/N15.jpg no/N16.jpg no/N17.jpg

In [ ]:
#!mv -t yes/ no/N1.JPG no/N15.jpg no/N16.jpg no/N17.jpg

# Sorting the Data

Images are labelet according to expected outputs 'yes' is labeled as 1, 'no' is labeled as 0.

Afterwards Data should be split as **test** and **training** data.

In [ ]:
# Define the path to the dataset
IMG_PATH = 'brain_tumor_dataset/'

# Create a list of all the image filenames
all_images = []
for folder in ['yes', 'no']:
    folder_path = os.path.join(IMG_PATH, folder)
    for filename in os.listdir(folder_path):
        
         all_images.append(os.path.join(folder_path, filename))

# Create a list of corresponding labels (0 for 'no', 1 for 'yes')
labels = [1 if 'Y' in filename else 0 for filename in all_images]

# Split the dataset into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(all_images, labels, test_size=0.04, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

# Print the sizes of the resulting datasets
print(f'Train set size: {len(X_train)}')
print(f'Validation set size: {len(X_val)}')
print(f'Test set size: {len(X_test)}')

Train set size: 181
Validation set size: 61
Test set size: 11


# Function for image preprocessing

Images dataset is argument for this funtion.

For every image it resizes the image with openCV and removes noise.

Finds contours and then gets the biggest contour in the image. Next step is to find extreme points (lowest bottom, highest top, right and left coordinates of contour). 

Final step is to cut the image with those extreme points so unnecessary area will be removed and the area of the interest is left, afterwards **returns** numpy array of images.

In [ ]:
def preprocess_images(images):
    preprocessed_images = []
    for i, img_path in enumerate(images):
        img = cv2.imread(img_path)
        img = cv2.resize(
            img,
            dsize=(224,224),
            interpolation=cv2.INTER_CUBIC
        )
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        # crop
        ADD_PIXELS = 0
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        new_img = cv2.resize(
            new_img,
            dsize=(224,224))
        preprocessed_images.append(new_img)
    return np.array(preprocessed_images)

# Apply the funtion

Preprocessing function is applyed to **training**, **validation** and **test** datasets


In [ ]:
# Apply the preprocessing to all the data subsets 
X_train_pre = preprocess_images(X_train)
X_val_pre = preprocess_images(X_val)
X_test_pre = preprocess_images(X_test)

In [ ]:
# Transform the subsets to numpy arrays 
X_train_pre_vgg = np.array([preprocess_input(image) for image in X_train_pre])
X_val_pre_vgg = np.array([preprocess_input(image) for image in X_val_pre])
X_test_pre_vgg = np.array([preprocess_input(image) for image in X_test_pre])

## Hyper Parameter Optimization (HPO) with W&B
- Create a W&B account at https://wandb.ai
then search for API key here: https://wandb.ai/settings
provide your API key when asked, in the next cell:
- install and import nessesary packages!

In [ ]:
!pip install -Uq wandb
import wandb
import pprint
import math
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

wandb.login()

True

## Rewrite your training code into two functions:

The "train" function contains mostly the code from the data_science_tutorial_may_5.ipynb
But you need to add the following parts:
- Possible hyperparameters as function arguments
- In this example we added "augmentation_multiplier" as hyperparameter,
it controlls the amount of variation in the augmentation step.
- clear_session() in the beginning, so that the model is overwritten every time the function is excecuted by the sweep agent
- wandb_callbacks for logging the temporal dynamics of the model during training.
- a final evaluation of the best model is being performed and provided as return values of the function



In [ ]:
def train(epochs=50, dropout=0.5,learning_rate=0.001, augmentation_multiplier=1.):

    # Clear old Keras session to start a new run
    clear_session()
    
    # Load the pre-trained VGG19 model
    base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224) + (3,)
    )

    # Create a new model by adding a few layers on top of the pre-trained model
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))

    # Freeze the weights of the pre-trained model
    model.layers[0].trainable = False

    # Compile the model with appropriate loss function, optimizer and metrics
    model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate),
    metrics=['accuracy']
    )

    # Print the summary of the model
    model.summary()

    # Define the training data generator with necessary data augmentation techniques
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20*augmentation_multiplier,
    width_shift_range=0.1*augmentation_multiplier,
    height_shift_range=0.1*augmentation_multiplier,
    shear_range=0.2*augmentation_multiplier,
    zoom_range=0.2*augmentation_multiplier,
    horizontal_flip=True,
    fill_mode='nearest'
    )

    # Create the training data generator using the training dataset and the data generator
    train_generator = train_datagen.flow(
    X_train_pre_vgg,
    y_train,
    batch_size=32
    )

    # Define a callback to reduce the learning rate when the validation accuracy plateaus
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=1e-5)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3,
                                restore_best_weights=True)

    # Define the validation data generator with appropriate data preprocessing
    val_datagen = ImageDataGenerator(rescale=1./255)

    # Create the validation data generator using the validation dataset and the data generator
    val_generator = val_datagen.flow(X_val_pre_vgg, y_val, batch_size=18)

    # Wandb callbacks are important for logging all training steps to wandb
    wandb_callbacks = [
        WandbMetricsLogger()
    ]


    # Train the model using the training data generator, validation data generator and the defined callbacks
    history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping, wandb_callbacks] # WANDB Callbacks added
    )

    # Evaluate best model on validation set without augmentation:
    results = model.evaluate(X_val_pre_vgg, np.array(y_val), batch_size=64)
    final_val_loss, final_val_accuracy = results[0], results[1]
    print(f'final_val_loss : {final_val_loss}')
    print(f'final_val_accuracy : {final_val_accuracy}')
    return final_val_loss, final_val_accuracy

The "sweep_train" function wraps the train so that it can load a different configuration every time a run is executet.
- you may whant to add additional meta data to the config, that is being common for all the runs that are going to be performed.
- then the "train" function is executet and evaluated
- finaly log the final results to W&B

In [ ]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "VGG19"
        wandb.config.dataset_name = "brain-tumor-mri"

        # initialize model
        final_val_loss, final_val_accuracy = train(
            wandb.config.epochs, 
            wandb.config.dropout,
            wandb.config.learning_rate,
            wandb.config.augmentation_multiplier)
        
        wandb.log({"final_val_loss": final_val_loss,
                   "final_val_accuracy": final_val_accuracy})

## Next, we are going to define the parameters to be investigated:
- We will use a random search for investigating the parameters.
- As the brain-tumor-mri dataset is very small, the model is very prone to overfitting.
- Therfore we are mostly interested in parameters controlling the regularization of the model.
- you can find more information about different search strategies here: https://docs.wandb.ai/guides/sweeps/define-sweep-configuration

In [ ]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'epochs': {'value': 50},
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.01},
    'augmentation_multiplier': {
        'distribution': 'uniform',
        'min': 0.0,
        'max': 2.0},
    'dropout': {'values': [0.3, 0.4, 0.5, 0.3, 0.7]}
    }

sweep_config['parameters'] = parameters_dict

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'augmentation_multiplier': {'distribution': 'uniform',
                                            'max': 2.0,
                                            'min': 0.0},
                'dropout': {'values': [0.3, 0.4, 0.5, 0.3, 0.7]},
                'epochs': {'value': 50},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.01,
                                  'min': 0.0001}}}


## Run the hyperparameter search.
- higher count means better exploration, but longer runtimes.
- in practice it is good to get a first impression of the parameter space and reducing it within an iterative approach



In [ ]:
sweep_id = wandb.sweep(sweep_config, project="DS_Tutorial_V1")
print(sweep_id)
wandb.agent(sweep_id, function=sweep_train, count=20)

Create sweep with ID: xqa6ct2t
Sweep URL: https://wandb.ai/nikofzj/DS_Tutorial_V1/sweeps/xqa6ct2t
xqa6ct2t


wandb: Agent Starting Run: cq1p10dm with config:
wandb: 	augmentation_multiplier: 0.06769023628139315
wandb: 	dropout: 0.4
wandb: 	epochs: 50
wandb: 	learning_rate: 0.00013659583789658145


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________
Epoch 1/50
6/6 [==============================] - 5s 597ms/step - loss: 0.8629 - accuracy: 0.4641 - val_loss: 0.6003 - val_accuracy: 0.7049 - lr: 1.3660e-04
Epoch 2/50
6/6

epoch/accuracy,▁▃▄▄▅▇▇▆▇▆██
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/learning_rate,████████▁▁▁▁
epoch/loss,█▆▅▅▄▃▂▃▁▂▁▁
epoch/lr,█████████▁▁▁
epoch/val_accuracy,▁▁▅██▇▆██▆▆▆
epoch/val_loss,█▆█▄▃▃▃▂▁▁▂▁
final_val_accuracy,▁
final_val_loss,▁
epoch/accuracy,0.78453
epoch/epoch,11


wandb: Agent Starting Run: zs4gshko with config:
wandb: 	augmentation_multiplier: 0.508191591770645
wandb: 	dropout: 0.4
wandb: 	epochs: 50
wandb: 	learning_rate: 0.0006399341601006144


...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________
Epoch 1/50
6/6 [==============================] - 5s 639ms/step - loss: 1.6511 - accuracy: 0.4862 - val_loss: 0.8901 - val_accuracy: 0.7049 - lr: 0.0029
Epoch 2/50
6/6 [==

epoch/accuracy,▁▅▅▅▆▇▇█▆▆▇▇██
epoch/epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
epoch/learning_rate,█████████▁▁▁▁▁
epoch/loss,█▄▅▃▃▂▂▂▂▃▂▂▁▁
epoch/lr,██████████▁▁▁▁
epoch/val_accuracy,▄▁▆▅█▆▆▇▇▆█▆█▇
epoch/val_loss,▆█▃▄▂▁▂▂▁▂▁▁▁▁
final_val_accuracy,▁
final_val_loss,▁
epoch/accuracy,0.79006
epoch/epoch,13


wandb: Agent Starting Run: uagl6sh5 with config:
wandb: 	augmentation_multiplier: 0.3743451752645621
wandb: 	dropout: 0.5
wandb: 	epochs: 50
wandb: 	learning_rate: 0.008480532635513384


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1)                 25089     
                                                                 
Total params: 20,049,473
Trainable params: 25,089
Non-trainable params: 20,024,384
_________________________________________________________________
Epoch 1/50
6/6 [==============================] - 4s 483ms/step - loss: 7.1560 - accuracy: 0.5635 - val_loss: 3.8592 - val_accuracy: 0.7049 - lr: 0.0085
Epoch 2/50
6/6 [==

epoch/accuracy,▁▁▄▄▅▆▅▅▇▇▇█▇▇
epoch/epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▂▂▂▂▂▁▁▁▁▁▁
epoch/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▃▃▁▄▅▆▅▄▇█▇▆██
epoch/val_loss,█▆▆▃▂▁▃▂▁▁▁▂▁▁
final_val_accuracy,▁
final_val_loss,▁
epoch/accuracy,0.89503
epoch/epoch,13
